In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset

import numpy as np

import math

import time

import dataloader
import models
import training_fun

import optuna

import joblib

In [8]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SEQ_LENGTH = 365 * 2
TARGET_SEQ_LENGTH = 365
BASE_LENGTH = SEQ_LENGTH - TARGET_SEQ_LENGTH

FORCING_DIM = 3

N_CATCHMENTS = 2346

# training hyperparameters
EPOCHS = 200
TRAIN_YEAR = 19
PATIENCE = 20

use_amp = True
compile_model = True

if compile_model:
    torch.set_float32_matmul_precision("high")

memory_saving = False
if memory_saving:
    storge_device = "cpu"
    computing_device = DEVICE
    VAL_STEPS = 500
else:
    storge_device = DEVICE
    computing_device = DEVICE

In [4]:
dtrain = dataloader.Forcing_Data(
    "data/data_train_w_missing.csv",
    record_length=7304,
    storge_device=storge_device,
    seq_length=SEQ_LENGTH,
    target_seq_length=TARGET_SEQ_LENGTH,
    base_length=BASE_LENGTH,
)

dval = dataloader.Forcing_Data(
    "data/data_val_w_missing.csv",
    record_length=4017,
    storge_device=storge_device,
    seq_length=SEQ_LENGTH,
    target_seq_length=TARGET_SEQ_LENGTH,
    base_length=BASE_LENGTH,
)

In [10]:
class Objective:
    def __init__(self, model_builder):
        self.model_builder = model_builder

    def objective(self, trial):

        val_losses = []

        # prepare early stopper
        early_stopper = training_fun.EarlyStopper(patience=PATIENCE, min_delta=0)

        # define model
        embedding, decoder = self.model_builder.define_model(trial)
        embedding, decoder = embedding.to(computing_device), decoder.to(
            computing_device
        )

        if compile_model:
            # pytorch2.0 new feature, complile model for fast training
            embedding, decoder = torch.compile(embedding), torch.compile(decoder)

        # define optimizers
        lr_embedding = trial.suggest_float("lr_embedding", 5e-5, 1e-2, log=True)
        embedding_optimizer = optim.Adam(embedding.parameters(), lr=lr_embedding)

        lr_decoder = trial.suggest_float("lr_decoder", 5e-5, 1e-2, log=True)
        decoder_optimizer = optim.Adam(decoder.parameters(), lr=lr_decoder)

        scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

        # define batch size
        batch_size_power = trial.suggest_int("batch_size_power", 5, 8)
        batch_size = 2**batch_size_power

        # train model
        for epoch in range(EPOCHS):

            # for each epoch get_random_batch method generates a batch that contains one year data for each catchment
            # repeat TRAIN_YEAR times to finish an epoch
            decoder.train()
            embedding.train()

            for year in range(TRAIN_YEAR):

                x_batch, y_batch = dtrain.get_random_batch()

                if memory_saving:
                    x_batch, y_batch = x_batch.to(computing_device), y_batch.to(
                        computing_device
                    )

                catchment_index = torch.randperm(
                    N_CATCHMENTS, device=computing_device
                )  # add randomness

                # interate over catchments
                for i in range(int(N_CATCHMENTS / batch_size)):

                    # prepare data
                    ind_s = i * batch_size
                    ind_e = (i + 1) * batch_size

                    selected_catchments = catchment_index[ind_s:ind_e]

                    x_sub, y_sub = x_batch[ind_s:ind_e, :, :], y_batch[ind_s:ind_e, :]

                    # prepare training, put the models into training mode
                    decoder_optimizer.zero_grad()
                    embedding_optimizer.zero_grad()

                    # forward pass
                    with torch.autocast(
                        device_type="cuda", dtype=torch.float16, enabled=use_amp
                    ):
                        code = embedding(selected_catchments)
                        out = decoder.decode(code, x_sub)

                        # backprop
                        loss = training_fun.mse_loss_with_nans(out, y_sub)

                    scaler.scale(loss).backward()
                    scaler.step(embedding_optimizer)
                    scaler.step(decoder_optimizer)
                    scaler.update()

            # validate model after each epochs
            decoder.eval()
            embedding.eval()

            # Handle pruning based on the intermediate value
            if memory_saving:
                val_loss = training_fun.val_model_mem_saving(
                    embedding=embedding,
                    decoder=decoder,
                    dataset=dval,
                    storge_device=storge_device,
                    computing_device=computing_device,
                    use_amp=use_amp,
                    val_metric=training_fun.mse_loss_with_nans,
                    return_summary=True,
                    val_steps=VAL_STEPS,
                )
            else:
                val_loss = (
                    training_fun.val_model(
                        embedding=embedding,
                        decoder=decoder,
                        dataset=dval,
                        storge_device=storge_device,
                        computing_device=computing_device,
                        use_amp=use_amp,
                        val_metric=training_fun.mse_loss_with_nans,
                        return_summary=True,
                    )
                    .detach()
                    .cpu()
                    .numpy()
                )

            trial.report(val_loss, epoch)

            if trial.should_prune():
                torch.cuda.empty_cache()
                raise optuna.exceptions.TrialPruned()

            # Early stop using early_stopper, break for loop
            if early_stopper.early_stop(val_loss):
                break

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        return early_stopper.min_validation_loss


In [11]:
LSTM_model_builder = training_fun.LSTM_model_builder(
    n_catchments = N_CATCHMENTS, base_length=BASE_LENGTH, forcing_dim=FORCING_DIM
)

LSTM_objective = Objective(LSTM_model_builder).objective

TCN_model_builder = training_fun.TCN_model_builder(
    n_catchments = N_CATCHMENTS, base_length=BASE_LENGTH, forcing_dim=FORCING_DIM
)
TCN_objective = Objective(TCN_model_builder).objective

In [12]:
study = optuna.create_study(
    study_name="base_model", direction="minimize", pruner=optuna.pruners.NopPruner()
)
study.optimize(LSTM_objective, n_trials=10)

joblib.dump(study, "base_LSTM_study.pkl")

[I 2022-12-06 23:40:38,611] A new study created in memory with name: base_model
[I 2022-12-06 23:41:16,941] Trial 0 finished with value: 8.105819702148438 and parameters: {'lstm_hidden_dim': 46, 'n_lstm_layers': 2, 'n_fc_layers': 2, 'LATENT_DIM_power': 1, 'drop_out_flag': True, 'dropout_rate': 0.3073757083157444, 'fc_dim0': 15, 'fc_dim1': 9, 'lr_embedding': 0.0014018063397661, 'lr_decoder': 0.0005248822900335671, 'batch_size_power': 6}. Best is trial 0 with value: 8.105819702148438.
[I 2022-12-06 23:41:45,364] Trial 1 finished with value: 8.43262004852295 and parameters: {'lstm_hidden_dim': 215, 'n_lstm_layers': 2, 'n_fc_layers': 3, 'LATENT_DIM_power': 1, 'drop_out_flag': False, 'fc_dim0': 14, 'fc_dim1': 11, 'fc_dim2': 15, 'lr_embedding': 8.046134687461299e-05, 'lr_decoder': 0.004481643847260491, 'batch_size_power': 7}. Best is trial 0 with value: 8.105819702148438.


In [13]:
study = optuna.create_study(
    study_name="base_model", direction="minimize", pruner=optuna.pruners.NopPruner()
)

study.optimize(TCN_objective, n_trials=10)

joblib.dump(study, "base_TCN_study.pkl")

[I 2022-12-06 23:43:00,909] A new study created in memory with name: base_model
[I 2022-12-06 23:43:15,640] Trial 0 finished with value: 8.626200675964355 and parameters: {'latent_dim_power': 2, 'kernel_size': 4, 'hidden_channel_dim': 1, 'drop_out_flag': True, 'dropout_rate': 0.10890623004313166, 'lr_embedding': 0.0035720874468074234, 'lr_decoder': 0.0008177612360125232, 'batch_size_power': 6}. Best is trial 0 with value: 8.626200675964355.
[I 2022-12-06 23:43:55,436] Trial 1 finished with value: 7.6127610206604 and parameters: {'latent_dim_power': 1, 'kernel_size': 5, 'hidden_channel_dim': 138, 'drop_out_flag': True, 'dropout_rate': 0.131800755417424, 'lr_embedding': 0.004662767612958916, 'lr_decoder': 5.420990594110336e-05, 'batch_size_power': 5}. Best is trial 1 with value: 7.6127610206604.
